In [31]:
# pip install -r requirements.txt

# Import


In [32]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import requests
import os
import time
from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Driver set up

## Browser settings


In [33]:
edge_options = webdriver.EdgeOptions()
# edge_options.add_argument('start-maximized')
edge_options.add_argument('disable-extensions')
edge_options.add_argument('ms:inPrivate')
edge_options.add_argument('disable-infobars')
edge_options.add_argument('inprivate')
edge_options.add_argument('--disable-notifications')
edge_options.add_argument('--enable-automation')
edge_options.add_experimental_option("detach", True)
edge_options.add_argument("--disable-features=msEdgeEnableNurturingFramework")
edge_options.add_experimental_option("prefs", {"user_experience_metrics": {"personalization_data_consent_enabled": True}})

## Secrets


In [34]:
my_email = 'your_user_name'
my_password = 'your_password'

## Initialize driver


In [35]:
driver = webdriver.Edge(keep_alive='./msedgedriver.exe', options=edge_options)
driver.get('https://mbasic.facebook.com')

# Function

## Parsing date


In [36]:
# Function to parse post timestamp
def parse_post_date(post):
    timestamp_element = post.find('abbr')
    timestamp_str = timestamp_element.text.strip()
    # Convert Vietnamese month names to English for datetime parsing
    month_map = {
        'tháng 1': 'January', 'tháng 2': 'February', 'tháng 3': 'March',
        'tháng 4': 'April', 'tháng 5': 'May', 'tháng 6': 'June',
        'tháng 7': 'July', 'tháng 8': 'August', 'tháng 9': 'September',
        'tháng 10': 'October', 'tháng 11': 'November', 'tháng 12': 'December'
    }
    for vietnamese, english in month_map.items():
        timestamp_str = timestamp_str.replace(vietnamese, english)

    timestamp_str = timestamp_str[0:timestamp_str.find('lúc')-1]
    
    post_date = datetime.strptime(timestamp_str, '%d %B')
    return post_date

## See more post


In [37]:


# Function to click on "Xem tin khác" link if available
def click_see_more_link():
    try:
        # Find the <a> tag containing the <span> with text "Xem tin khác"
        see_more_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[span[text()='Xem tin khác']]")))
        see_more_link.click()
        time.sleep(2)  # Wait for posts to load
    except NoSuchElementException:
        print("No 'Xem tin khác' link found")
    except Exception as e:
        print(f"Error clicking on 'Xem thêm' link: {str(e)}")

## Reaction count


In [38]:
src_react_list = ['https://scontent.fsgn5-10.fna.fbcdn.net/m1/v/t6/An-OY9MLHL-ZSjdewcBHqsUqbG_iKP7YPdv-VtLFuTjPb6JeVsW1ptvkRgGXI8lL3mIfyuCjIzzN5JF-_YVzwnU8ZKYcYb0PKaeIHGfC0HZDZMTjHQ.png?_nc_eui2=AeGpHRF1csrkKXGi14tWMJ8VvjDbL2sEdxq-MNsvawR3GhocIfjsr4KCCEhpavnwbnZfnQ_l-5uJBhT1XVQNRCMU&ccb=10-5&oh=00_AYCzLOj9WzFQ3yDmIHcYmAKTDb4b7vpBYyviGuFyMjFGGQ&oe=66A289AA&_nc_sid=7da55a', 'https://scontent.fsgn5-10.fna.fbcdn.net/m1/v/t6/An-6QMzYRPEP4s4LQEshZQ6Bx40DD5EIgaWfC5G5LQixNEFZf-hRWdy4--zN6aq5ZV-ry-7lMXUy7c9tyL_ZouBwtWg60RZ6s1A6ICUdO2-qJm1R.png?_nc_eui2=AeGdhxwh1SOsd-sFEx4lY0U3qciSfxI3tqOpyJJ_Eje2ox8_HW8fxUa08kIqyQBP630o5FZCPNJJtSZDkV1ISYaF&ccb=10-5&oh=00_AYCORDXA8-d1EXLff-YdFWSvFa8YZ7wkIVqtCJ7OkSfUxA&oe=66A282FE&_nc_sid=7da55a', 'https://scontent.fsgn5-10.fna.fbcdn.net/m1/v/t6/An-VBmEj0LPhn7r9Xz5guezTJlexmqlwQ-3VYyM27u4DWR7mpQVnzOLz67OONCtekVvQJcNDlQxkBMluKsBslk_OVGsOR14sE4Ws08M62Zieu5U.png?_nc_eui2=AeHpvc-ZZRXjSXNwIl9B1IJuvKB6rGfP5lC8oHqsZ8_mUOA_Wj6RW-Wna6SgVqZHGUTy0YIm2gUQFrQs2NrC0M_O&ccb=10-5&oh=00_AYAU6mbllxUctwhUrZ-T5tXpiYrhLo2x0TUlHNxJhPUKWg&oe=66A26581&_nc_sid=7da55a', 'https://scontent.fsgn5-10.fna.fbcdn.net/m1/v/t6/An9NamgJaDr3DddUsHGQ-envu3mt13Yo_osqqzTDZsaDqAup9-IzNhhOTMYkiqHYJHwCpwAuy7UWZlYUrLtp48435Lr-yNPScoTUkcxZkDKgdlVf.png?_nc_eui2=AeE-_5Al0Elz7b_rMJ-5nP6SIhlIWl90TrAiGUhaX3ROsMe7BEAw8Ln6iiJifOtL7gibdZrYUzWnVYVCszJn7a5M&ccb=10-5&oh=00_AYAc5I9U16Sh1QG5pLlkSg3xqxV4kqR3zrRoRpVwjgR6Lw&oe=66A27BC6&_nc_sid=7da55a', 'https://scontent.fsgn5-10.fna.fbcdn.net/m1/v/t6/An9ltC9_wCT7IwUJcodFBHhRbLYkvfoGweZkoXPe1V1EHIKN-qZQDxkIdv01AFtBOginaTV9YyHM-5WgTV0SFOQ10z32oD0llybf3laTMDTZXYpIvS02.png?_nc_eui2=AeEaCj3YRTEzq0vhUo7XKSbNlBpYHxmoj4WUGlgfGaiPhWDxN-xKyaWG7gNcDuwtVntH_CWbkzJ7nRDeXWNr2psG&ccb=10-5&oh=00_AYDVnhtP5LvY519badHMM36NX4F9_m_U9Bj6eD-bSPiWnw&oe=66A2624F&_nc_sid=7da55a', 'https://scontent.fsgn5-10.fna.fbcdn.net/m1/v/t6/An-B1pSuUzVYjvgsMKX_Vqj4-_lLytkM6Nozgd-KEQtGeI8Y9Dj67ingsy3_SeWpSSuVqtkvm86Rb2Nn5PDEyd2W0StC1YTvqRttUA4Ffuh_FPpP.png?_nc_eui2=AeFviAdyt8yDBjYRV4jLPeK-IP4eemdD7CYg_h56Z0PsJowSxVK9ax6kCuFXTpmnpcfcNn01PreB6bNAWkSP0tim&ccb=10-5&oh=00_AYAI7UrgLszeyfpcX4LQ6EZW1rRBj0xBjPyqR3_IAGxAWQ&oe=66A282FF&_nc_sid=7da55a', 'https://scontent.fsgn5-10.fna.fbcdn.net/m1/v/t6/An_awEcP5a-VJkiSKC4SklmLyo8p7Q3iP5vL6HDsa_ZTJdFfRRdtUFNJfr9LXPYfMhVSkFk4hqLRcj3zU9hTsyzpPGIc4jC3fiqwidCEo8AGZ4Rq.png?_nc_eui2=AeEUnQMs_BsOrH0ihfcpedXkgFb2z8QtW-iAVvbPxC1b6F1rLK8ZKzV7NDlgCR5PHdh0KjEBqisqSCPHpO-2BcZz&ccb=10-5&oh=00_AYC9w3V3YUiNJsn_4WJWS8YZVKDOT-9FD4U1mjAyaweTzA&oe=66A28512&_nc_sid=7da55a']

def extract_reaction_count(post, src_list):
    texts = []
    top_react =[]
    for img_tag in post.find_all('img'):
        src = img_tag.get('src')
        if src in src_list:
            # Traverse up the parent tree to find <a> tag
            parent = img_tag.parent
            while parent:
                if parent.name == 'a':
                    texts.append(parent.get_text(strip=True).replace('.',''))
                    break
                parent = parent.parent
        if src == src_list[0]:
            top_react.append('Sad')
        elif src == src_list[1]:
            top_react.append('Heart')
        elif src == src_list[2]:
            top_react.append('Haha')
        elif src == src_list[3]:
            top_react.append('Wow')
        elif src == src_list[4]:
            top_react.append('Love')
        elif src == src_list[5]:
            top_react.append('Angry')
        elif src == src_list[6]:
            top_react.append('Like')
    return texts, top_react

## Comment count


In [39]:
def extract_comment_count(post):
    comment_count = 0
    # Tìm tất cả các thẻ <a> trong bài post
    a_tags = post.find_all('a')
    # Duyệt qua từng thẻ <a> để tìm số lượng bình luận
    for a_tag in a_tags:
        # Lấy văn bản bên trong thẻ <a>
        text = a_tag.get_text(strip=True)
        # Kiểm tra xem văn bản có chứa từ "bình luận" không
        if 'bình luận' in text:
            # Trích xuất số lượng bình luận từ văn bản và chuyển đổi thành số nguyên
            try:
                comment_count = int(text.split()[0].replace('.', ''))
                break  # Dừng khi đã tìm thấy số bình luận
            except ValueError:
                pass  # Bỏ qua nếu không thể chuyển đổi thành số nguyên
    return comment_count

# Crawling flow

## Login


In [40]:
# Login to Facebook
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='email']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='pass']")))

username.clear()
username.send_keys(my_email)
password.clear()
password.send_keys(my_password)

login_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[type='submit']"))).click()
other_time_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[class='bo bp bq br bt']"))).click()

driver.get('https://mbasic.facebook.com/TCCCVN/')
timeline_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[class='ct']"))).click()

click_see_more_link()

## Crawling


In [41]:
found_end_post = False
post_content = []
data = []

# List of possible class names for posts
post_classes = ['bm bn bo', 'bn bo bp']

while not found_end_post:
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')

    posts_found = False

    # Try to find posts using each class name in the list
    for post_class in post_classes:
        posts = soup.find_all('div', class_=post_class)
        if posts:
            for post in posts:
                timestamp = parse_post_date(post)
                if timestamp:
                    if timestamp.month <= 1:  # Stop if the post is from March or earlier
                        found_end_post = True
                        break
                    
                    # Extract paragraphs from each post
                    paragraphs_list = []
                    paragraphs = post.find_all('p')
                    for p in paragraphs:
                        paragraph_text = p.get_text(strip=True)
                        paragraphs_list.append(paragraph_text)
                    
                    post_content.append(paragraphs_list)
                    react, top_react = extract_reaction_count(post, src_react_list)
                    comment_count = extract_comment_count(post)
                    # Append data to list for dataframe
                    data.append({'date': timestamp.strftime('%d %B'), 'content': paragraphs_list, 'reaction_count': react, 'top_react': top_react, 'comment_count': comment_count})
                    
                    print('Post' + " " + timestamp.strftime('%d %B'))
                    print(react)
                    print(top_react)
                    # print(timestamp.strftime('%d %B'))
                    # print(paragraphs_list)
                    # print('-' * 50)
                    
                    posts_found = True  # Indicate that posts were found
            
            if found_end_post:
                break

    if not posts_found:
        print("No more posts found matching the criteria.")
        break

    if not found_end_post:
        click_see_more_link()

# # Close the WebDriver
# driver.quit()

Post 14 June
['44']
['Like']
Post 14 June
['1410', '1410']
['Like', 'Heart']
Post 12 June
['156', '156', '156']
['Like', 'Heart', 'Wow']
Post 10 June
['55']
['Like']
Post 08 June
['102', '102']
['Like', 'Heart']
Post 08 June
['84', '84', '84']
['Like', 'Heart', 'Wow']
Post 07 June
['78', '78']
['Like', 'Heart']
Post 05 June
['91', '91', '91']
['Like', 'Heart', 'Wow']
Post 29 May
['148', '148']
['Like', 'Heart']
Post 28 May
['80', '80']
['Like', 'Heart']
Post 27 May
['8581', '8581']
['Like', 'Heart']
Post 24 May
['88', '88', '88']
['Like', 'Heart', 'Wow']
Post 23 May
['6474', '6474']
['Like', 'Heart']
Post 22 May
['602', '602']
['Like', 'Heart']
Post 21 May
['107', '107']
['Like', 'Heart']
Post 20 May
['31', '31', '31']
['Like', 'Heart', 'Wow']
Post 14 May
['165', '165', '165']
['Like', 'Heart', 'Wow']
Post 10 May
['99', '99']
['Like', 'Heart']
Post 06 May
['376', '376']
['Like', 'Heart']
Post 04 May
['149', '149']
['Like', 'Heart']
Post 03 May
['799', '799']
['Like', 'Heart']
Post 02 M

# Data transformation


In [42]:
# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)

# Change date format to 'dd/mm'
df['date'] = pd.to_datetime(df['date'], format='%d %B')
df = df[df['date'].dt.month < 5]
# df['date'] = df['date'].dt.strftime('%d/%m')

df = df.reset_index(drop=True)
# df = df.drop_duplicates(subset='date', keep='first')

In [43]:
# Create an empty list to store titles
titles = []

# Iterate through each row in 'content' column
for content in df['content']:
    # Get the first element of the list (assuming 'content' is a list of strings)
    title = content[0] if isinstance(content, list) and len(content) > 0 else None
    titles.append(title)

# Assign the list of titles to a new column 'title' in the dataframe
df['title'] = titles

In [44]:
# Sắp xếp lại DataFrame theo thứ tự giảm dần của date
df = df.sort_values(by='date', ascending=True).reset_index(drop=True)

# Tính khoảng cách giữa các ngày
df['day_diff'] = df['date'].diff().dt.days

df['date'] = df['date'].dt.strftime('%d/%m')

# Chuyển đổi danh sách các số thành các giá trị dạng số
df['reaction_count'] = df['reaction_count'].apply(lambda x: list(map(int, x)))

# Giữ lại các giá trị duy nhất trong từng danh sách
df['reaction_count'] = df['reaction_count'].apply(set)

# Chuyển đổi set thành danh sách hoặc lấy giá trị đầu tiên
df['reaction_count'] = df['reaction_count'].apply(lambda x: next(iter(x)))

In [45]:
df.to_csv('coke_data.csv', index=False)